In [ ]:
pip install pypdf

In [ ]:
from openai import AzureOpenAI
import os
from IPython.display import  Markdown

client = AzureOpenAI(
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.environ.get("OPENAI_KEY"),
  api_version="2023-12-01-preview"
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import TextLoader,PyPDFLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings, AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory


In [37]:
txt_file_path = '../docs/Phone-List-Schools.pdf'
loader = PyPDFLoader(file_path=txt_file_path)
data = loader.load()

In [38]:
data

[Document(page_content=' \n \n \n  \n \n                      \n                                 \n \n                                  \n \n \n \n Lake Washington School District \nDelivery:  16250 NE 74th Street, Redmond 98052  Mail:  PO Box 97039, Redmond WA  98073-9739\n \nElementary Schools Phone # Elementary Schools Phone # \nAlcott\t 4213 - 228th Avenue NE, Redmond 98053 936-2490\n \nP: Jon Hedin OM: Carolina Borrego                              \nAP: Barbara Deming \nAudubon\t 3045 - 180th Avenue NE, Redmond 98052 936-2500\n P: Kimo Spray  OM: Joan Livesey\n AP: Ian Maver\n \nBell \t 11212 NE 112th Street, Kirkland 98033 936-2510\n \nP: Heidi Paul OM: Maddy Sanders \nPreschool AP: Anne Balicki \nBlackwell\t 3225 - 205th Place NE, Sammamish 98074 936-2520\n P: Jim Eaton OM: Linda Kramer                                    \nAP: Angela Morrison\n \nCarson\t 1035 244th Avenue NE, Sammamish 98074  936-2750\n P: Scott Power OM: Julie Henderson                                         

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [ ]:
embeddings = AzureOpenAIEmbeddings(api_key=os.environ.get("OPENAI_KEY") ,api_version= "2023-12-01-preview",azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),azure_deployment="text-embedding-ada-002")

In [ ]:
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [ ]:
llm = AzureChatOpenAI(model="gpt-4",api_key=os.environ.get("OPENAI_KEY") ,api_version= "2023-12-01-preview",azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"))

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

In [35]:
query = "Redmond school location and phone number?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer
Markdown(answer)

Here are the locations and phone numbers of the schools in Redmond within the Lake Washington School District:

**Elementary Schools:**

- Alcott
  4213 - 228th Avenue NE, Redmond 98053
  Phone: 936-2490
  
- Audubon
  3045 - 180th Avenue NE, Redmond 98052
  Phone: 936-2500

- Dickinson / Explorer (The address is for Dickinson, which houses Explorer)
  7040 - 208th Avenue NE, Redmond 98053
  Phone for Dickinson: 936-2530
  Phone for Explorer: 936-2533
  
- Einstein
  18025 NE 116th Street, Redmond 98052
  Phone: 936-2540
  
- Mann
  17001 NE 104th Street, Redmond 98052
  Phone: 936-2610
  
- Redmond
  16800 NE 80th Street, Redmond 98052
  Phone: 936-2660
  
- Rockwell
  11125 - 162nd Avenue NE, Redmond 98052
  Phone: 936-2670
  
- Rosa Parks
  22845 NE Cedar Park Crescent, Redmond 98053
  Phone: 936-2650
  
- Rush
  6101 - 152nd Avenue NE, Redmond 98052
  Phone: 936-2690

**Middle Schools:**

- Evergreen
  6900 208th Avenue NE, Redmond 98053
  Phone: 936-2320
  
- Redmond
  10055 - 166th Avenue NE, Redmond 98052
  Phone: 936-2440

**High Schools:**

- Redmond
  17272 NE 104th Street, Redmond 98052
  Phone: 936-1800

**Other Programs:**

- Tesla STEM
  4301 228th Ave NE, Redmond 98053
  Phone: 936-2770

- Transition Academy - Redmond
  16315 NE 87th ST, Suite B-1, Redmond 98052
  Phone: 861-3452

Please note that for some schools such as Dickinson/Explorer, they share the same address but have different phone numbers.

In [ ]:
query = "List all the schools in Redmond in pointers"
result = conversation_chain({"question": query})
answer = result["answer"]
answer
Markdown(answer)

In [36]:
query = "List all the schools starting with letter F"
result = conversation_chain({"question": query})
answer = result["answer"]
answer
Markdown(answer)

The schools in Redmond that start with the letter "F" are:

1. Finn Hill Middle School
   Location: 8040 NE 132nd Street, Kirkland 98034 (Finn Hill is within the Lake Washington School District and serves the Redmond area, but is physically located in Kirkland.)
   Phone Number: 936-2340

2. Frost Elementary School
   Location: 11801 NE 140th Street, Kirkland 98034 (Frost is within the Lake Washington School District and serves the Redmond area, but is physically located in Kirkland.)
   Phone Number: 936-2560

3. Futures School
   Location: 10601 NE 132nd Street, Kirkland 98034 (Futures School is within the Lake Washington School District and serves the Redmond area, but is physically located in Kirkland.)
   Phone Number: 936-1635

Please note that while these schools are part of the Lake Washington School District and may serve students in the Redmond area, their physical addresses are listed in Kirkland.